# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import os
import numpy as np
import math
import tensorflow as tf

from time import time
from keras.callbacks import TensorBoard

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

from numpy.random import seed
from tensorflow import set_random_seed


C:\Users\controlslab\Anaconda3\envs\gpu_tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0.2

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(12, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(16, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(12, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc3'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(0.1), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models
    model = RULmodel_SN_5(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model




In [6]:
num_features = len(selected_features)

shape = len(selected_features)*window_size
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)


# Load Data

In [7]:
tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 129

In [8]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0.2)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 129. Cros-Validation ratio 0.2
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(14547, 420)
(14547, 1)
Cross-Validation data (X, y)
(20, 420)
(20, 1)
Testing data (X, y)
(100, 420)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.55400697 -0.34526854 -0.63551968 ... -0.63636364  0.62711864
   0.43246698]
 [-0.60278746 -0.18107417 -0.56084763 ... -0.63636364  0.74576271
   0.48047152]
 [-0.79094077 -0.11611253 -0.32552977 ... -0.81818182  0.40677966
   0.43843204]
 [-0.70034843 -0.32941176 -0.44177598 ... -0.45454545  0.47457627
   0.56000568]
 [-0.66550523 -0.59130435 -0.18183653 ... -0.81818182  0.23728814
   0.59892061]]
[[129.]
 [129.]
 [129.]
 [129.]
 [129.]]
Cross-Validation data (X, y)
[[-0.23344948  0.12071611 -0.31745711 ... -0.45454545  0.44067797
   0.19471666]
 [-0.24738676  0.32890026  0.50393542 ... -0.27272727 -0.08474576
   0.0123562 ]
 [-0.1080

# Train the model and test some Tunable Model functionalities

In [9]:
class TrainValTensorBoard(TensorBoard):
    def __init__(self, log_dir='./logs', **kwargs):
        
        # Make the original `TensorBoard` log to a subdirectory 'training'
        training_log_dir = os.path.join(log_dir, 'training')
        super(TrainValTensorBoard, self).__init__(training_log_dir, **kwargs)

        # Log the validation metrics to a separate subdirectory
        self.val_log_dir = os.path.join(log_dir, 'validation')

    def set_model(self, model):
        # Setup writer for validation metrics
        self.val_writer = tf.summary.FileWriter(self.val_log_dir)
        
        super(TrainValTensorBoard, self).set_model(model)

    def on_epoch_end(self, epoch, logs=None):
        # Pop the validation logs and handle them separately with
        # `self.val_writer`. Also rename the keys so that they can
        # be plotted on the same figure with the training metrics
        logs = logs or {}
        #print("Logs:", logs.items())
        val_logs = {k.replace('val_', ''): v for k, v in logs.items() if k.startswith('val_')}
        #print("Val_logs:", val_logs)
        for name, value in val_logs.items():
            #print("Name:", name)
            #print("Value:", value)
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.val_writer.add_summary(summary, epoch)
        self.val_writer.flush()

        # Pass the remaining logs to `TensorBoard.on_epoch_end`
        logs = {k: v for k, v in logs.items() if not k.startswith('val_')}
        super(TrainValTensorBoard, self).on_epoch_end(epoch, logs)

    def on_train_end(self, logs=None):
        super(TrainValTensorBoard, self).on_train_end(logs)
        self.val_writer.close()


In [10]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = nFeatures*tModel.data_handler.sequence_length


model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')


#tensorboard = TensorBoard(log_dir='./logs')
#log_dir="./logs".format(time())
tensorboard = TrainValTensorBoard(log_dir="./logs".format(time()),write_graph=False)

tModel.epochs = 100 
tModel.train_model(learningRate_scheduler=lrate, verbose=1, tensorboard=tensorboard,)




training with cv
Train on 14547 samples, validate on 20 samples
Epoch 1/100
 9728/14547 [===================>..........] - ETA: 0s - loss: 8683.0142 - mean_squared_error: 8672.8291 ETA: 0s - loss: 8799.3220 - mean_squared_error: 8789. - ETA: 0s - loss: 8777.3061 - mean_squared_error: 8 - ETA: 0s - loss: 8679.6016 - mean_squared_error: 8669.

C:\Users\controlslab\Anaconda3\envs\gpu_tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.100725). Check your callbacks.
  % delta_t_median)


10752/14547 [=====================>........] - ETA: 0s - loss: 8687.5622 - mean_squared_error: 8677.37 - ETA: 0s - loss: 8658.2398 - mean_squared_error: 8648.0548

C:\Users\controlslab\Anaconda3\envs\gpu_tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.119690). Check your callbacks.
  % delta_t_median)
C:\Users\controlslab\Anaconda3\envs\gpu_tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.147618). Check your callbacks.
  % delta_t_median)


12288/14547 [========================>.....] - ETA: 0s - loss: 8637.1676 - mean_squared_error: 8626.9727

...

11776/14547 [=======================>......] - ETA: 0s - loss: 203.6879 - mean_squared_error: 183.7039 ETA: 0s - loss: 202.7438 - mean_squared_error: 182.75

C:\Users\controlslab\Anaconda3\envs\gpu_tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.110977). Check your callbacks.
  % delta_t_median)
C:\Users\controlslab\Anaconda3\envs\gpu_tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.110481). Check your callbacks.
  % delta_t_median)


14336/14547 [============================>.] - ETA: 0s - loss: 204.4038 - mean_squared_error: 184.4265 ETA: 0s - loss: 204.2770 - mean_squared_error: 184. - ETA: 0s - loss: 204.1104 - mean_squared_error: 184.1344

C:\Users\controlslab\Anaconda3\envs\gpu_tensorflow\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.128915). Check your callbacks.
  % delta_t_median)


14547/14547 [==============================] - 4s 250us/step - loss: 203.9918 - mean_squared_error: 184.0166 - val_loss: 306.8349 - val_mean_squared_error: 286.9239


In [11]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=True)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

20/20 [==============================] - 0s 50us/step
scores
Engine 1, Predicted RUL 145.05886840820312, Rounded RUL 145.0, Real RUL 112.0
Engine 2, Predicted RUL 22.499370574951172, Rounded RUL 22.0, Real RUL 98.0
Engine 3, Predicted RUL 5.530709266662598, Rounded RUL 5.0, Real RUL 69.0
Engine 4, Predicted RUL 97.9443359375, Rounded RUL 97.0, Real RUL 82.0
Engine 5, Predicted RUL 52.59850311279297, Rounded RUL 52.0, Real RUL 91.0
Engine 6, Predicted RUL 112.5205307006836, Rounded RUL 112.0, Real RUL 93.0
Engine 7, Predicted RUL 7.716517925262451, Rounded RUL 7.0, Real RUL 91.0
Engine 8, Predicted RUL 87.5880126953125, Rounded RUL 87.0, Real RUL 95.0
Engine 9, Predicted RUL 129.9507598876953, Rounded RUL 129.0, Real RUL 111.0
Engine 10, Predicted RUL 79.93858337402344, Rounded RUL 79.0, Real RUL 96.0
Engine 11, Predicted RUL 55.984073638916016, Rounded RUL 55.0, Real RUL 97.0
Engine 12, Predicted RUL 11.1906156539917, Rounded RUL 11.0, Real RUL 124.0
Engine 13, Predicted RUL 113.331184

In [12]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=False)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 4ms/step: 
scores
Engine 1, Predicted RUL 112.33728790283203, Rounded RUL 112.0, Real RUL 112.0
Engine 2, Predicted RUL 128.4075164794922, Rounded RUL 128.0, Real RUL 98.0
Engine 3, Predicted RUL 65.3319320678711, Rounded RUL 65.0, Real RUL 69.0
Engine 4, Predicted RUL 88.33563995361328, Rounded RUL 88.0, Real RUL 82.0
Engine 5, Predicted RUL 104.48941802978516, Rounded RUL 104.0, Real RUL 91.0
Engine 6, Predicted RUL 110.60025787353516, Rounded RUL 110.0, Real RUL 93.0
Engine 7, Predicted RUL 96.71713256835938, Rounded RUL 96.0, Real RUL 91.0
Engine 8, Predicted RUL 94.51876831054688, Rounded RUL 94.0, Real RUL 95.0
Engine 9, Predicted RUL 122.18558502197266, Rounded RUL 122.0, Real RUL 111.0
Engine 10, Predicted RUL 91.20944213867188, Rounded RUL 91.0, Real RUL 96.0
Engine 11, Predicted RUL 97.83984375, Rounded RUL 97.0, Real RUL 97.0
Engine 12, Predicted RUL 94.43685913085938, Rounded RUL 94.0, Real RUL 124.0
Engine 13, Predicted RUL 75.